# Empleos_IT

El proposito de este proyecto verificar l factibilidad de hacer scraping en este sitio

- https://empleosti.com.mx/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get update
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
from datetime import date
from time import sleep
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import pandas as pd
import requests
from urllib import request

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('disable-infobars')
chrome_options.add_argument("window-size=1200x600")

In [ ]:
#Diccionario de grados academicos
diccionario_academic_degree = {
    #Espanol
    #La ingenieria tambien es una licenciatura
    r"((L|l)icenciatura|(I|i)ngenier(i|í)a|(L|l)icenciad(o|a)\s(en|de)|(I|i)ngenier(o|a)\s(en|de)|\sLic\.\s|\sIng\.\s)" : "Licenciatura", 
    r"(M|m)aestr(i|í)a" : "Maestria", 
    r"(D|d)octorado" : "Doctorado", 
    #English
    r"((B|b)achelor('s)?|\sB(\.)?S(\.)?\sin)" : "Bachelor", 
    r"((M|m)aster('s)?|\sM(\.)?S(\.)?(\s|\/P(H|h)(\.)?D)(or|in))" : "Master", 
    r"P(H|h)(\.)?D" : "PhD"
}

#Diccionario de Soft Skills
#Creo que hay pocos soft skill en español ya que la mayoria de las descripcion estaban en ingles
diccionario_soft_skill ={
    #Spanish
    r"\b(A|a)nal(í|i)tica\b" : "Capacidad Analitica", 
    r"\b((C|c)reatividad|(C|c)reativ(a|o)s)\b" : "Creatividad",
    r"\b((E|e)mpuje\spor\sobtener\sresultados\b|(E|e)nfocado\sa\sresultados)\b" : "Enfocado a resultados",
    r"\b(M|m)anejo\sde\sconflictos\b" : "Manejo de conflictos",
    r"\b(I|i)nspirar\b" :"Inspirar",
    r"\b((O|o)rganizado|(O|o)rganizas)\b" : "Organizado",
    r"\b(P|p)roactiv(a|o)" : "Proactivo",
    r"\b(R|r)esoluci(ó|o)n\sde\sproblemas\b" : "Resolucion de problemas",
    r"\b(T|t)rabajo\sen\sequipo\b" : "Trabajo en equipo",
    r"\b(T|t)rabajar?\sde\sforma\sindependiente\b" : "Trabaja de forma independiente", 
    #English
    r"\b((A|a)nalytical|(A|a)nalytics?)\b" : "Analitical", 
    r"\b(A|a)ttention\sto\sdetails?\b" : "Attention to details", 
    r"\b(C|c)oach\b" : "Coach", 
    r"\b(C|c)ollaborat(e|es|ing|ion|ively)\b" : "Collaborate", 
    r"\b(C|c)ommunicat(e|es|ion|ing)\b" : "Comunication skills",
    r"\b((C|c)ontinuously\s(L|l)earn|(L|l)earning|(L|l)earn)\b" : "Continuously learn", 
    r"\b(C|c)reativity\b" : "Creativity", 
    r"\b(C|c)uriosity\b" : "Curiosity",
    r"\b(C|c)ritical\s(T|t)hinking\b" : "Critical Thinking", 
    r"\b(D|d)etail(\-|\s)(O|o)riented\b"
    r"\b(D|d)ealing\s(well)?\swith\sambiguity\b" : "Dealing well with ambiguity",
    r"\b(D|d)ynamic\s(E|e)nvironment\b":"Dynamic enviroment",
    r"\b(E|e)fficiency\b" : "Efficiency",
    r"\b(E|e)nthusiasm\b" : "Enthusiasm", 
    r"\b(G|g)roup\sintegration\b" : "Group integration",
    r"\b(G|g)rowth\s(M|m)indset\b" : "Growth Mindset",
    r"\b(H|h)igh\s(S|s)tandars\b" : "High Standars",
    r"\b((I|i)ndependent\s(W|w)orker|(W|w)orks?\s(I|i)ndependently)\b" : "Independent Worker",
    r"\b((I|i)nnovati(on|ve)|(I|i)nnovators)\b" : "Innovation",
    r"\b(I|i)nvolved\b" : "Involved",
    r"\b(I|i)nterpersonal\b" : "Interpersonal", 
    r"\b((L|l)eadership|(L|l)ead\sa\steam)\b" : "Leadership",
    r"\b(M|m)anaging\s\w*\steams\b" : "Managing teams",
    r"\b(M|m)entor\b" : "Mentor",
    r"\b(M|m)otivated\b" : "Motivated",
    r"\b(M|m)ultifunctional\stasking\b" : "Multifunctional tasking",
    r"\b(O|o)rganization(al)?\b" : "Organization",
    r"\b(P|p)lanification\b" : "Planification",
    r"\b(P|p)ositive\sattitude\b" : "Positive attitude",
    r"\b(P|p)rioritizing\sneeds\b" : "Prioritizing needs",
    r"\b(P|p)roactivity\b" : "Proactivity",
    r"\b((P|p)roblem(\-|\s)(solving|(S|s)olver))\b" : "Problem solving",
    r"\b(S|s)elf(\s|\-)directed\b" : "Self directed", 
    r"\b(S|s)elf(\s|\-)management\b" : "Self Management", 
    r"\b(S|s)elf(\s|\-)starter\b" : "Self Starter", 
    r"\b(S|s)trategic\b" : "strategic",
    r"\b(T|t)arget(\s)?[-](\s)?(B|b)ased\s(R|r)esults\b" : "Target – Based results",
    r"\b(T|t)eam\s(P|p)layer" : "Team Player",
    r"\b(T|t)eam(\s)?(W|w)ork" : "Team Work", 
    r"\b((W|w)ritten|(V|v)erbal|(O|o)ral)(\sand\s|\/|\s[&]\s)((W|w)ritten|(V|v)erbal|(O|o)ral)\b" : "Written and oral communication",
    r"\b(cross(\-|\s)functional\s(teams|partners)|(I|i)nterdisciplinary|multiple\sfunctional\sgroups)\b" : "Working with cross functional teams"
}

#Diccionario de Hard skills
diccionario_HardSkill = {
    #learn the basics
    r"\bhtml5?\b" : "HTML", 
    r"\bcss3?\b" : "CSS", 
    r"\bjavascript\b" : "JavaScript", 
    r"\bes\s2020[+]\b" : "ES 2020+",
    #javascript Task runners
    r"\bnpm\sscripts\b" : "npm Scripts",
    r"\bgulp\b" : "Gulp",
    r"\bgrunt\b" : "Grunt", 
    #javascript woking with linux
    r"\benvironment\b" : "Environment", 
    r"\bterminal\b" : "Terminal", 
    #javascript version control
    r"\bgit\b" : "Git", 
    r"\bgithub\b" : "Github", 
    r"\bbit\sbucket\b" : "Bit bucket",
    #javascript module bundler
    r"\bwebpack\b" : "Webpack", 
    r"\brollup\b" : "Rollup",
    #javascript testing
    r"\bjest\b" : "Jest",
    r"\bpuppeteer\b" : "Puppeteer",
    r"\bcupress\b" : "Cypress",
    #javascript libraries and framework
    r"\breact\b" : "React", 
    r"\bangular\b" : "Angular", 
    r"\bvue\b" : "Vue",
    r"\bnuxt(\.js)?\b" : "Nuxt.js",
    r"\bsvelte\b" : "Svelte",
    r"\bredux\b" : "Redux", 
    r"\bxstate\b" : "Xstate",
    r"\brecoil\b" : "Recoil",
    r"\bnext\b" : "Next", 
    r"\bgatsby\b" : "Gatsby",
    r"\benzyme\b" : "Enzime",
    r"\bjest\b" : "Jest",
    #css
    r"\bflexbox\b" : "Flexbox",
    r"\bcss\sgrid\b" : "CSS grid",
    #css pre processors
    r"\bsass\b" : "SASS",
    r"\bpostcss\b" : "PostCSS",
    #css framework
    r"\bstyled\scomponents\sreact\b" : "Styled Components React",
    r"\btailwind\scss\b" : "TailWind Css",
    r"\bbulma\b" : "Bulma",
    r"\bbootstraps\b" : "Bootstraps",
    #CI/CD
    r"\bgithub\sactions\b" : "Github Actions",
    #Deployment
    r"\bs3\b" : "S3",
    r"\bsurge\.sn\b" : "Surge.sn",
    r"\bvercel\b" : "Vercel",
    r"\bfirebase\b" : "Firebase",
    r"\bnetlify\b" : "Netlify", 
    #Essentials
    r"\blinux\b" : "Linux",
    r"\bgit\b" : "Git",
    r"\bgithub\b" : "Github",
    #Languages
    r"\bc[#]\b" : "c#",
    r"\bgo\b" : "Go",
    r"\b\.net\b" : ".NET",
    r"\bjava\b" : "Java", 
    r"\bspring\sframework\b" : "Spring Framework",
    r"\bhibernate\b" : "Hibernate",
    r"\bjavascript\b" : "JavaScript",
    r"\bnode(\.js)?\b" : "Node.js",
    r"\bexpress\b" : "Express",
    r"\bfastify\b" : "Fastify",
    r"\bpython\b" : "Python",
    r"\bflask\b" : "Flask",
    r"\bdjango\b" : "Django",
    #Database
    r"\bsql\b" : "SQL",
    r"\bmysql\b" : "MySQL",
    r"\bpostgresql\b" : "PostgreSQL",
    r"\boracle\b" : "Oracle",
    r"\b(mssql|sql\sserver)\b" : "MSSQL",
    r"\bnosql\b" : "NoSQL",
    r"\bmongodb\b" : "MongoDB",
    r"\bdynamodb\b" : "DynamoDB",
    r"\bcassandra\b" : "Cassandra", 
    r"\binmemory\b" : "Inmemory", 
    r"\bredis\b" : "Redis", 
    #Storage
    r"\bawss3\b" : "AWSS3",
    r"\b(azure\s)?blob\sstorage\b" : "Azure Blob Storage",
    #Services and APIs
    r"\bresful\sapis?\b" : "Restful APIs",
    r"\bmicroservices\sarchitecture\b" : "Microservices Architecture",
    r"\bgraphql\b" : "GraphQL",
    #Develops and deployment
      #containerization orchestration
    r"\bdocker\b" : "Docker",
    r"\bkubernetes\b" : "Kubernetes",
    r"\bdevops\b" : "Devops",
    r"\baws\b" : "AWS",
    r"\bazure\b" : "Azure",
    r"\bdigital\socean\b" : "Digital Ocean",
    r"\bheroku\b" : "HerokU",
    r"\bnetlify\b" : "Netlify",
    r"\baws\slambda\b" : "AWS Lambda",
    r"\bazure\sfunction\b" : "Azure Function",
    r"\bservreless\sframework\b" : "Serverless Framework",
    r"\baws\scdk\b" : "AWS CDK",
    #CI/CD
    r"\bgithub\sactions\b" : "Github Actions",
    r"\bjenikins(\sx)?\b" : "Jenikins",
    r"\bcircle\sci\b" : "Circle CI",
    #security
    r"\bowasp\b" : "OWASP",
    r"\bprotective\sservices\s(and|[&])\sdatabase\sagainst\sthreats\b" : "Protective Services and database against threats"
}

#Diccionario de idiomas
diccionario_languages = {    
    #Ingles
    r"\b((A|a)dvanced?\s(or\sfluent\s)?(in\s)?(E|e)nglish|C1)\b" : "Advanced English",   
    r"\b(I|i)ntermediate(\s?\-\s?(A|a)dvanced?)?\s(E|e)nglish\b" : "Intermediate English",
    r"\b(F|f)luent\sin\s(w+\sand\s)?(E|e)nglish\b" : "Fluent in English",
    r"\b((B|b)iling(ü|u)e|(B|b)i(\s|\-)(L|l)ingual)\b" : "Bilingue",
    #Español
    r"\b(I|i)ngl(e|é|é)s[:]?\s((I|i)ntermedio[-])?(A|a)vanzado\b" : "Ingles Avanzado",
    r"\b(I|i)ngl(e|é|é)s\st(e|é)cnico\b" : "Ingles Tecnico",
    r"\b(I|i)ngl(e|é|é)s[:]?\s((N|n)ivel\s)?(I|i)ntermedio\b" : "Ingles Intermedio",
    r"\b(I|i)ngl(e|é|é)s[:]?\s((N|n)ivel\s(I|i)ntermedio\s)?(A|a)lto\b" : "Ingles Avanzado",
    #Algunas descripciones no especificaban nivel, solo mencionaban que se necesitaba ingles
    r"\b(I|i)ngl(e|é|é)s\b" : "Ingles",
    r"\bEnglish\b" : "English",
    r"\bItalian\b": "Italian",
    r"\bPortuguese\b" : "Portuguese",
    r"\bSpanish\b" : "Spanish"
}

#Patron de years of experience 
busqueda_years_experience = r"((\+?([0-4]?[0-9]|[1-9])\+?(\s?[-]\s?|\sto\s))?\+?([0-4]?[0-9]|[1-9])\+?\s((Y|y)ears?|(A|a)(ñ|ñ)os?))"
patron_year = re.compile(busqueda_years_experience)


In [ ]:
keys_degree = diccionario_academic_degree.keys()
keys_soft_skill = diccionario_soft_skill.keys()
keys_hardSkill = diccionario_HardSkill.keys()
keys_idiomas = diccionario_languages.keys()

def ObtenerGradoEducativo(df, columna_descripcion):
  resultados_degree = []
  for i in range(len(df)):
    resultado_fila_degree = ""
    for key in keys_degree:  
      pattern = re.compile(key)
      #print( type(df[columna_descripcion][i]) )
      try:
        if pattern.search(df[columna_descripcion][i]):
          resultado_fila_degree = resultado_fila_degree + diccionario_academic_degree[key] + ", " 
      except:
        pass
    #Resultados grado academico
    resultado_fila_degree = resultado_fila_degree[0:-2]
    resultados_degree.append(resultado_fila_degree)  

  return resultados_degree

def ObtenerIdiomas(df, columna_descripcion):
  resultados_idiomas = []
  for i in range(len(df)):
    resultado_fila_idioma = ""
    for key in keys_idiomas:  
      pattern = re.compile(key)
      #print( type(df[columna_descripcion][i]) )
      try:
        if pattern.search(df[columna_descripcion][i]):
          resultado_fila_idioma = resultado_fila_idioma + diccionario_languages[key] + ", " 
      except:
        pass
    #Resultados grado academico
    resultado_fila_idioma = resultado_fila_idioma[0:-2]
    resultados_idiomas.append(resultado_fila_idioma)  

  return resultados_idiomas

def ObtenerYears(df, columna_descripcion):
  resultados_years= []
  for i in range(len(df)):
    resultado_fila_years_experience = ""
    try:
      if patron_year.search(df[columna_descripcion][i]):
        match_year = patron_year.findall(df[columna_descripcion][i])
        for m in match_year:
          resultado_fila_years_experience = resultado_fila_years_experience + m[0] + ", "
    except:
      pass
    resultado_fila_years_experience = resultado_fila_years_experience[0:-2]
    resultados_years.append(resultado_fila_years_experience) 
  return resultados_years

def ObtenerSoftSkill(df, columna_descripcion):
  resultados_soft_skill = []
  for i in range(len(df)):
    resultado_fila_soft_skill = ""
    for key in keys_soft_skill:
      pattern = re.compile(key)
      try:
        if pattern.search(df[columna_descripcion][i]):
          resultado_fila_soft_skill = resultado_fila_soft_skill + diccionario_soft_skill[key] + ", "
      except:
        pass
    #Resultados de fila Soft skill
    resultado_fila_soft_skill = resultado_fila_soft_skill[0:-2]
    resultados_soft_skill.append(resultado_fila_soft_skill)  

  return resultados_soft_skill

def ObtenerSkill(df, columna_descripcion):
  resultados_skill= []
  for i in range(len(df)):
    resultado_fila_skill = ""
    for key in keys_hardSkill:
      pattern = re.compile(key, re.IGNORECASE)
      try:
        if pattern.search(df[columna_descripcion][i]):
              resultado_fila_skill = resultado_fila_skill + diccionario_HardSkill[key] + ", "
      except: 
        pass
    #Resultados skill Front End
    resultado_fila_skill = resultado_fila_skill[0:-2]
    resultados_skill.append(resultado_fila_skill)  

  return resultados_skill

In [ ]:
#Patron de remote
busqueda_remote = r"remot(o|e)"
patron_remote= re.compile(busqueda_remote, re.IGNORECASE)

def isRemote(df, columna_info):
  resultados_remote= []
  for i in range(len(df)):
    resultado_fila_remote = ""
    try:
      if patron_remote.search(df[columna_info][i]):
        resultado_fila_remote = "Remote"
      else:
        resultado_fila_remote = None
    except:
      pass
    resultados_remote.append(resultado_fila_remote) 
  return resultados_remote

In [ ]:
estados_Mexico ={
    r'aguascalientes' : "Aguascalientes",  r'baja\scalifornia' : 'Baja California', 
    r'baja\scalifornia\ssur' : 'Baja California Sur',  r'campeche' : 'Campeche', 
    r'chiapas':'Chiapas', r'chihuahua' :'Chihuahua', 
    r'ciudad\sde\sm(é|e)xico' : 'Ciudad de Mexico', 
    r'coahuila' : 'Coahuila', r'colima' : 'Colima', 
    r'durango' : 'durango', r"estado\sde\sm(é|e)xico" : "Estado de mexico",
    r'guanajuato' : 'Guanajuato', r'guerrero' : 'Guerrero', 
    r'hidalgo' : 'Hidalgo', r'jalisco' : 'Jalisco', 
    r'michoac(á|a)n' : 'Michoacan', r'morelos' : 'Morelos', 
    r'nayarit' : 'Nayarit', r'nuevo\sLe(ó|o)n' : "Nuevo Leon", 
    r'oaxaca' : "Oaxaca", r'puebla' : "Puebla", r'quer(é|e)taro' : 'Queretaro', 
    r'quintana\sroo' : "Quintana Roo", r'san\sluis\spotos(í|i)': 'San Luis Potosí', 
    r'sinaloa' : 'Sinaloa', r'sonora' : 'Sonora', 
    r'tabasco' : 'Tabasco', r'tamaulipas' : 'Tamaulipas', 
    r'tlaxcala' : 'Tlaxcala', r'veracruz' : 'Veracruz', 
    r'yucat(á|a)n' : 'Yucatan', r'zacatecas' : 'Zacatecas', 
    r"(remot(o|e)|trabaj(a|o)\sdesde\scasa)" : "Remoto"}

keys_estados = estados_Mexico.keys()
def ObtenerEstado(df, columna):
    estados = df[columna].str.split(",").str.get(1)
    for i in range (len(estados)):
        if(type(estados[i]) == float):
            for key in keys_estados:
                pattern = re.compile(key, re.IGNORECASE)
                try:
                    if pattern.search(df[columna][i]):
                        estados[i] = estados_Mexico[key]
                        continue
                except: 
                    continue
    return estados


In [ ]:
def ExtraerInfo_Empleos_TI():
    Title = ""
    Location = ""
    Company = ""
    Salary = ""
    Tipo_Puesto = ""
    English = ""
    Description = ""
    Logo = ""
    #Obtener el nombre del puesto
    try:
        title_driver = driver.find_element_by_xpath('//h1[@class="section-title"]')
        Title = title_driver.text
    except:
        Title = None
    
    #Locacion, tipo de puesto, english and salary 
    try:
        datos_driver = driver.find_elements_by_xpath('//ul[@class="field-set-vacancy"]//p[@class="gray-subarea-title"]')
        try:
            Salary = datos_driver[0].text
        except:
            Salary = None

        try:
            Location = datos_driver[1].text
        except:
            Location = None

        try:
            Tipo_Puesto = datos_driver[2].text
        except:
            Tipo_Puesto = None

        try:
            English = datos_driver[3].text
        except:
            English = None    
    except :
        Salary = None
        Location = None
        Tipo_Puesto = None
        English = None  
    #Nombre de la compania
    try:
        company_driver = driver.find_element_by_id('searchable-enterprise')
        Company = company_driver.text
    except :
        Company = None
    #Descripcion
    try:
        description_driver = driver.find_element_by_id('vacancy-description')
        Description = description_driver.text
    except:
        Description = None



    resultados = {"Title" : Title, "Location" : Location, "Company" : Company, "Description" : Description,  "Salary" : Salary, "Tipo_Puesto" : Tipo_Puesto, "English" : English}

    return resultados

In [1]:
#Ingresa tu nombre para obtener los terminos que te tocan
persona = "Cecy"
if(persona == "Mario"):
  #Mario
  terminos = ["Java", ".Net/C#", "JavaScript", "PHP", "AngularJS", "ReactJS", "Python", "Diseño de software", "Android"]
elif(persona == "Ian"):
  #Ian
  terminos = ["IOS", "PLSQL", "Ruby", "Cobol", "C++", "Testing", "SAP", "Analista de sistemas", "Administracion de proyectos"]
elif(persona == "Jezrrel"):
  #Jezrrel
  terminos = ["Redes", "Auditoria de seguridad", "Ciberseguridad", "Pentesting", "Sysadmin", "Devops", "Administrador de base de datos", "Business intelligence", "Data Warehouse"]
elif(persona == "Cecy"):
  #Cecy
  terminos = ["Big data", "Data Science", "Sales force", "UI UX", "MKT digital", "SEO", "SEM", "Ventas TI", "Reclutamiento TI"]

In [2]:
terminos

['Big data',
 'Data Science',
 'Sales force',
 'UI UX',
 'MKT digital',
 'SEO',
 'SEM',
 'Ventas TI',
 'Reclutamiento TI']

In [ ]:
#Ingresar el termino que se usara en el cuadro de busqueda

#test
#terminos = ["SEO"]

#Creo el dataframe
df_Empleos_TI = pd.DataFrame(columns = ["Title", "Location", "Company" ,"Description", "Salary", "Tipo_Puesto", "English"])
driver_path = "chromedriver"
driver = webdriver.Chrome(executable_path=driver_path,options=chrome_options)

#Recorrer la lista
for termino in terminos:
  #pagina principal
  driver.get("https://empleosti.com.mx/empleos?k=&l=")
  #cuadro de busqueda
  element = driver.find_element(By.XPATH,'//input[@id="input-k"]')
  #ingresar el nombre de la company
  element.send_keys(termino)
  #enter
  element.send_keys(Keys.ENTER)
  #espera
  WebDriverWait(driver, 10)
  try:
    next = driver.find_element(By.PARTIAL_LINK_TEXT, 'chevron_right')  
    #driver.get(next.get_attribute("href"))
    url_base = next.get_attribute("href")
    url_base = url_base[:-1]
    
  except:
    url_base = driver.current_url
  print(driver.current_url)
  #Revisa cuantas paginas hay
  paginas = driver.find_elements(By.XPATH,'//a[@class="page-link"]')
  try:
    ultima_pagina = int(paginas[-1].text)
  except:
    pass
  empleos_pagina = driver.find_elements(By.XPATH,'//div[@class="noContainerUp container"]//ul[@class="job-eti"]//li')  
  #Obtener datos de la primera pagina
  lista_empleos = driver.find_elements(By.XPATH,'//div[@class="noContainerUp container"]//ul[@class="job-eti"]//a')       
  lista =[]

  for job in lista_empleos:
      lista.append(job.get_attribute("href"))

  for link in lista:
      #print(link)
      driver.get(link)
      sleep(2)
      df_Empleos_TI = df_Empleos_TI.append(ExtraerInfo_Empleos_TI(), ignore_index=True)

  if(len(paginas)>=2):
    for i in range (2,ultima_pagina + 1):
      url_completo = url_base + str(i)
      print(url_completo)
      driver.get(url_completo)
      #datos de demas paginas
      lista_empleos = driver.find_elements(By.XPATH,'//div[@class="noContainerUp container"]//ul[@class="job-eti"]//a')       
      lista =[]
      
      for job in lista_empleos:
          lista.append(job.get_attribute("href"))

      for link in lista:
          #print(link)
          driver.get(link)
          sleep(2)
          df_Empleos_TI = df_Empleos_TI.append(ExtraerInfo_Empleos_TI(), ignore_index=True)



https://empleosti.com.mx/empleos?k=Big+data


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


https://empleosti.com.mx/empleos?query=Big%20data&l=&k=Big%20data&ms=&mxs=&jk=&jd=&page=2
https://empleosti.com.mx/empleos?query=Big%20data&l=&k=Big%20data&ms=&mxs=&jk=&jd=&page=3
https://empleosti.com.mx/empleos?query=Big%20data&l=&k=Big%20data&ms=&mxs=&jk=&jd=&page=4
https://empleosti.com.mx/empleos?query=Big%20data&l=&k=Big%20data&ms=&mxs=&jk=&jd=&page=5
https://empleosti.com.mx/empleos?query=Big%20data&l=&k=Big%20data&ms=&mxs=&jk=&jd=&page=6
https://empleosti.com.mx/empleos?query=Big%20data&l=&k=Big%20data&ms=&mxs=&jk=&jd=&page=7
https://empleosti.com.mx/empleos?query=Big%20data&l=&k=Big%20data&ms=&mxs=&jk=&jd=&page=8
https://empleosti.com.mx/empleos?query=Big%20data&l=&k=Big%20data&ms=&mxs=&jk=&jd=&page=9
https://empleosti.com.mx/empleos?query=Big%20data&l=&k=Big%20data&ms=&mxs=&jk=&jd=&page=10
https://empleosti.com.mx/empleos?query=Big%20data&l=&k=Big%20data&ms=&mxs=&jk=&jd=&page=11
https://empleosti.com.mx/empleos?query=Big%20data&l=&k=Big%20data&ms=&mxs=&jk=&jd=&page=12
https:/

In [ ]:
df_Empleos_TI

,Title,Location,Company,Description,Salary,Tipo_Puesto,English,Academy_grade,Years_Experience,SoftSkill,HardSkill,Remoto,Estado,Accessed_Date
0,Data Architecht,Remoto - Solo México,Pypol Soluciones,Nos encontramos en búsqueda de un Data Archite...,"$ 50,000 a 60,000",Empleado de tiempo completo,Nivel Intermedio,,,,,Remote,Remoto,2021-10-26
1,Arquitecto Google Cloud Platform,"Miguel Hidalgo, Ciudad de México",Pypol Soluciones,Estamos buscando un Arquitecto Google Cloud Pl...,"$ 45,000 a 50,000",Empleado de tiempo completo,Nivel Intermedio,,1 año,,,None,Ciudad de México,2021-10-26
2,Data Analyst,"Miguel Hidalgo, Ciudad de México",Pypol Soluciones,Nos encontramos en búsqueda de un Data Analyst...,"$ 35,000 a 40,000",Empleado de tiempo completo,Nivel Intermedio,Licenciatura,2-3 años,,SQL,None,Ciudad de México,2021-10-26
3,Consultor SSAS,"Miguel Hidalgo, Ciudad de México",Pypol Soluciones,Nos encontramos en búsqueda de un Consultor SS...,"$ 35,000 a 40,000",Empleado de tiempo completo,Nivel Intermedio,,3 años,,SQL,None,Ciudad de México,2021-10-26
4,Desarrollador Big Data,"Miguel Hidalgo, Ciudad de México",Pypol Soluciones,Nos encontramos en búsqueda de Desarrolladores...,"$ 45,000 a 50,000",Empleado de tiempo completo,Nivel Intermedio,Licenciatura,,,"Java, Python, Oracle, NoSQL, Cassandra",None,Ciudad de México,2021-10-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,Project Manager (Inglés Avanzado),"Venustiano Carranza, Ciudad de México",CAPITAL EMPRESARIAL HORIZONTE SA DE CV,Capital Empresarial Horizonte Empresa Mexicana...,"$ 15,000 a 20,000",Empleado de tiempo completo,Nivel Avanzado,Licenciatura,3 años,"Capacidad Analitica, Organizado, Trabajo en eq...",,None,Ciudad de México,2021-10-26
1114,63-1759 Jefe de Turno,"Miguel Hidalgo, Ciudad de México",Involve,¡Prepárate para formar parte de un gran equipo...,No especificado,Empleado de tiempo completo,Nivel Básico,,,Creatividad,,None,Ciudad de México,2021-10-26
1115,Arquitecto SCCM,"Benito Juárez, Ciudad de México",CAPITAL EMPRESARIAL HORIZONTE SA DE CV,Capital Empresarial Horizonte Empresa Mexicana...,"$ 50,000 a 60,000",Empleado de tiempo completo,Nivel Avanzado,Licenciatura,3 años,,,None,Ciudad de México,2021-10-26
1116,DevOps (Inglés Conversacional),"Benito Juárez, Ciudad de México",CAPITAL EMPRESARIAL HORIZONTE SA DE CV,Capital Empresarial Horizonte Empresa Mexicana...,"$ 45,000 a 50,000",Empleado de tiempo completo,Nivel Avanzado,Licenciatura,3 años,,"Git, Docker, Kubernetes, Devops, Azure",None,Ciudad de México,2021-10-26


In [ ]:
"""Obtener grado educativo, años de experiencia, english, soft skills, 
skills front y back end a traves de expresiones regulares """
df_Empleos_TI["Academy_grade"] = ObtenerGradoEducativo(df_Empleos_TI, "Description") 
df_Empleos_TI["Years_Experience"] = ObtenerYears(df_Empleos_TI, "Description")
df_Empleos_TI["SoftSkill"] = ObtenerSoftSkill(df_Empleos_TI, "Description") 
df_Empleos_TI["HardSkill"] = ObtenerSkill(df_Empleos_TI, "Description") 


In [ ]:
#Columnas que aparecen en stackoverflow, pero aqui no
df_Empleos_TI["Remoto"] = isRemote(df_Empleos_TI, "Location")
#df_Empleos_TI["Visa_sponsor"] = None
#df_Empleos_TI["Paid_relocation"] = None
#Columnas que aparecen en indeed, pero aqui no
#df_Empleos_TI["ID"] = None
df_Empleos_TI["Estado"] = ObtenerEstado(df_Empleos_TI, "Location")
#df_Empleos_TI["Synopsis"] = None
#df_Empleos_TI["Post_Date"] = None
df_Empleos_TI["Accessed_Date"] = date.today()
#Columnas que aparecen en Glassdoor, pero aqui no
#df_Empleos_TI["Min_Salary"] = None
#df_Empleos_TI["Max_Salary"] = None
#df_Empleos_TI["Pay_Period"] = None
#df_Empleos_TI["Company_Overview"] = None

In [ ]:
# Save file
df_Empleos_TI.to_csv('/content/drive/MyDrive/Colab Notebooks/Empleos_TI/data/Empleos_TI_{}_{}.csv'.format(persona, date.today().strftime('%Y%m%d')), index=False)